In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error

# Parameters
fir_order = 250
low_cutoff = 0.01
high_cutoff = 0.1
Fs = 8.138  # Sampling frequency in Hz
butterworth_order = 4

# Design FIR filter (Hamming window)
nyq = 0.5 * Fs
low = low_cutoff / nyq
high = high_cutoff / nyq
b_fir = signal.firwin(fir_order + 1, [low, high], window='hamming', pass_zero=False)

# Design Butterworth filter
b_butter, a_butter = signal.butter(butterworth_order, [low, high], btype='band')

def process_file(file_path):
    # Load the data
    data = loadmat(file_path)
    normalized_data = data['normalized_data']

    # Apply FIR filter
    fir_filtered = signal.filtfilt(b_fir, [1], normalized_data.T).T

    # Apply Butterworth filter
    butterworth_filtered = signal.filtfilt(b_butter, a_butter, normalized_data.T).T

    # Calculate metrics
    mse_fir = np.mean([mean_squared_error(normalized_data[:, i], fir_filtered[:, i]) for i in range(normalized_data.shape[1])])
    mse_butter = np.mean([mean_squared_error(normalized_data[:, i], butterworth_filtered[:, i]) for i in range(normalized_data.shape[1])])

    def rmse(original, filtered):
        return np.sqrt(mean_squared_error(original, filtered))

    def snr(original, filtered):
        signal_power = np.mean(original ** 2)
        noise_power = np.mean((original - filtered) ** 2)
        return 10 * np.log10(signal_power / noise_power)

    def spectral_distortion(original, filtered):
        return np.mean(np.abs(np.fft.rfft(original) - np.fft.rfft(filtered)))

    def thd(signal):
        harmonics = np.fft.rfft(signal)
        thd_value = np.sum(np.abs(harmonics[1:])**2) / np.abs(harmonics[0])**2
        return thd_value

    rmse_fir = np.mean([rmse(normalized_data[:, i], fir_filtered[:, i]) for i in range(normalized_data.shape[1])])
    rmse_butter = np.mean([rmse(normalized_data[:, i], butterworth_filtered[:, i]) for i in range(normalized_data.shape[1])])

    snr_fir = np.mean([snr(normalized_data[:, i], fir_filtered[:, i]) for i in range(normalized_data.shape[1])])
    snr_butter = np.mean([snr(normalized_data[:, i], butterworth_filtered[:, i]) for i in range(normalized_data.shape[1])])

    corr_fir = np.mean([np.corrcoef(normalized_data[:, i], fir_filtered[:, i])[0, 1] for i in range(normalized_data.shape[1])])
    corr_butter = np.mean([np.corrcoef(normalized_data[:, i], butterworth_filtered[:, i])[0, 1] for i in range(normalized_data.shape[1])])

    spectral_dist_fir = np.mean([spectral_distortion(normalized_data[:, i], fir_filtered[:, i]) for i in range(normalized_data.shape[1])])
    spectral_dist_butter = np.mean([spectral_distortion(normalized_data[:, i], butterworth_filtered[:, i]) for i in range(normalized_data.shape[1])])

    thd_fir = np.mean([thd(fir_filtered[:, i]) for i in range(normalized_data.shape[1])])
    thd_butter = np.mean([thd(butterworth_filtered[:, i]) for i in range(normalized_data.shape[1])])

    return {
        'mse_fir': mse_fir,
        'mse_butter': mse_butter,
        'rmse_fir': rmse_fir,
        'rmse_butter': rmse_butter,
        'snr_fir': snr_fir,
        'snr_butter': snr_butter,
        'corr_fir': corr_fir,
        'corr_butter': corr_butter,
        'spectral_dist_fir': spectral_dist_fir,
        'spectral_dist_butter': spectral_dist_butter,
        'thd_fir': thd_fir,
        'thd_butter': thd_butter,
    }

# Process all relevant files
folder_path = '/content/drive/MyDrive/removedspikes_nonan_replace_window10avg/ply_dtrnd/tddr/norm'
results = []

for filename in os.listdir(folder_path):
    if filename.endswith('.mat') and 'rest' in filename:
        file_path = os.path.join(folder_path, filename)
        print(f"Processing file: {filename}")
        result = process_file(file_path)
        results.append(result)

# Calculate average results
avg_results = {key: np.mean([r[key] for r in results]) for key in results[0].keys()}

# Print results
print("\nAverage results across all processed files:")
print(f"FIR filter (Hamming window):")
print(f"  Average MSE: {avg_results['mse_fir']:.6f}")
print(f"  Average RMSE: {avg_results['rmse_fir']:.6f}")
print(f"  Average SNR: {avg_results['snr_fir']:.2f} dB")
print(f"  Average Correlation: {avg_results['corr_fir']:.4f}")
print(f"  Average Spectral Distortion: {avg_results['spectral_dist_fir']:.6f}")
print(f"\nButterworth filter:")
print(f"  Average MSE: {avg_results['mse_butter']:.6f}")
print(f"  Average RMSE: {avg_results['rmse_butter']:.6f}")
print(f"  Average SNR: {avg_results['snr_butter']:.2f} dB")
print(f"  Average Correlation: {avg_results['corr_butter']:.4f}")
print(f"  Average Spectral Distortion: {avg_results['spectral_dist_butter']:.6f}")

# Plot frequency responses
w, h_fir = signal.freqz(b_fir)
w, h_butter = signal.freqz(b_butter, a_butter)

plt.figure(figsize=(12, 6))
plt.semilogx(w * Fs / (2 * np.pi), 20 * np.log10(abs(h_fir)), label='FIR (Hamming)')
plt.semilogx(w * Fs / (2 * np.pi), 20 * np.log10(abs(h_butter)), label='Butterworth')

plt.title('Filter Frequency Responses')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [dB]')
plt.grid(True)
plt.axvline(low_cutoff, color='green', linestyle='--', label='Cutoff frequencies')
plt.axvline(high_cutoff, color='green', linestyle='--')
plt.legend()
plt.xlim(0.01, Fs/2)
plt.ylim(-60, 10)
plt.show()

# Additional comparison plots (MSE, RMSE, SNR, Correlation, Spectral Distortion)
labels = ['MSE', 'RMSE', 'SNR', 'Correlation', 'Spectral Distortion']
fir_values = [
    avg_results['mse_fir'], avg_results['rmse_fir'], avg_results['snr_fir'],
    avg_results['corr_fir'], avg_results['spectral_dist_fir']
]
butter_values = [
    avg_results['mse_butter'], avg_results['rmse_butter'], avg_results['snr_butter'],
    avg_results['corr_butter'], avg_results['spectral_dist_butter']
]

x = np.arange(len(labels))
width = 0.35  # Width of bars

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, fir_values, width, label='FIR (Hamming)')
bars2 = ax.bar(x + width/2, butter_values, width, label='Butterworth')

ax.set_xlabel('Metrics')
ax.set_title('Comparison of FIR (Hamming) and Butterworth Filters')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Adding value labels on top of bars
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

add_labels(bars1)
add_labels(bars2)

plt.show()

# Separate plot for THD
labels_thd = ['THD']
fir_thd_values = [avg_results['thd_fir']]
butter_thd_values = [avg_results['thd_butter']]

x_thd = np.arange(len(labels_thd))
width_thd = 0.35  # Width of bars

fig, ax = plt.subplots(figsize=(6, 6))
bars1_thd = ax.bar(x_thd - width_thd/2, fir_thd_values, width_thd, label='FIR (Hamming)')
bars2_thd = ax.bar(x_thd + width_thd/2, butter_thd_values, width_thd, label='Butterworth')

ax.set_xlabel('Metrics')
ax.set_title('Total Harmonic Distortion (THD) Comparison')
ax.set_xticks(x_thd)
ax.set_xticklabels(labels_thd)
ax.legend()

# Adding value labels on top of bars
def add_labels_thd(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

add_labels_thd(bars1_thd)
add_labels_thd(bars2_thd)

plt.show()
